In [1]:
using JuMP, HiGHS
using Plots; plotly()
using DataFrames, CSV

┌ Warning: Failed to load integration with PlotlyBase & PlotlyKaleido.
│   exception =
│    ArgumentError: Package PlotlyKaleido not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyKaleido")` to install the PlotlyKaleido package.
│    Stacktrace:
│      [1] macro expansion
│        @ .\loading.jl:1163 [inlined]
│      [2] macro expansion
│        @ .\lock.jl:223 [inlined]
│      [3] require(into::Module, mod::Symbol)
│        @ Base .\loading.jl:1144
│      [4] top-level scope
│        @ C:\Users\sound\.julia\packages\Plots\Ec1L1\src\backends.jl:569
│      [5] eval
│        @ .\boot.jl:368 [inlined]
│      [6] _initialize_backend(pkg::Plots.PlotlyBackend)
│        @ Plots C:\Users\sound\.julia\packages\Plots\Ec1L1\src\backends.jl:567
│      [7] backend(pkg::Plots.PlotlyBackend)
│        @ Plots C:\Users\sound\.julia\packages\Plots\Ec1L1\src\backends.jl:245
│      [8] #plotly#324
│        @ C:\Users\sound\.julia\packages\Plots\Ec1L1\src\backends.jl:86 [inlined]
│      [9] 

In [ ]:
# import Pkg; Pkg.add("PlotlyBase")

In [5]:
generators = CSV.read("./Western_interconnect_full_run_10d/Generators_data.csv", DataFrame)
generators = select(generators, :R_ID, :Resource, :technology,:Zone, :THERM, :VRE, :STOR, :HYDRO,
                    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_Size, :New_Build, :Max_Cap_MW,
                    :Inv_Cost_per_MWyr, :Fixed_OM_Cost_per_MWyr, :Inv_Cost_per_MWhyr, :Fixed_OM_Cost_per_MWhyr,
                    :Var_OM_Cost_per_MWh, :Start_Cost_per_MW, :Start_Fuel_MMBTU_per_MW, :Heat_Rate_MMBTU_per_MWh, :Fuel,
                    :Min_Power, :Ramp_Up_Percentage, :Ramp_Dn_Percentage, :Up_Time, :Down_Time,
                    :Eff_Up, :Eff_Down);
  # Set of all generators
G = generators.R_ID;
# Uncomment this line to explore the data if you wish:
# show(generators, allrows=true, allcols=true)


In [15]:
generators

Row,R_ID,Resource,technology,Zone,THERM,VRE,STOR,HYDRO,Commit,Existing_Cap_MW,Existing_Cap_MWh,Cap_Size,New_Build,Max_Cap_MW,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,Inv_Cost_per_MWhyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh,Start_Cost_per_MW,Start_Fuel_MMBTU_per_MW,Heat_Rate_MMBTU_per_MWh,Fuel,Min_Power,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,Eff_Up,Eff_Down
,Int64,String,String,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,String31,Float64,Float64,Float64,Int64,Int64,Float64,Float64
1,1,CA_N_batteries_1,Batteries,1,0,0,1,0,0,2613.1,19019.5,56.81,-1,0.0,0,6235,0,0,0.0,0,0.0,10.34,None,0.0,1.0,1.0,0,0,0.92,0.92
2,2,CA_N_biomass_1,Biomass,1,0,0,0,0,0,250.4,0.0,2.09,-1,0.0,0,150850,0,0,6.87,0,0.0,15.99,None,0.314,1.0,1.0,0,0,1.0,1.0
3,3,CA_N_conventional_hydroelectric_1,Conventional Hydroelectric,1,0,0,0,1,0,7519.1,0.0,57.4,-1,0.0,0,49259,0,0,0.0,0,0.0,10.34,None,0.058,0.5,0.5,0,0,1.0,1.0
4,4,CA_N_geothermal_1,Geothermal,1,0,0,0,0,0,757.8,0.0,31.57,-1,0.0,0,218925,0,0,0.0,0,0.0,10.34,None,0.347,1.0,1.0,0,0,1.0,1.0
5,5,CA_N_hydroelectric_pumped_storage_1,Hydroelectric Pumped Storage,1,0,0,1,0,0,2276.9,35292.0,94.87,-1,0.0,0,49259,0,0,0.0,0,0.0,10.34,None,0.236,1.0,1.0,0,0,0.866,0.866
6,6,CA_N_natural_gas_fired_combined_cycle_1,Natural Gas Fired Combined Cycle,1,1,0,0,0,1,3112.8,0.0,119.72,0,0.0,0,17267,0,0,4.76,96,2.0,8.68,pacific_naturalgas,0.492,0.64,0.64,6,6,1.0,1.0
7,7,CA_N_natural_gas_fired_combined_cycle_2,Natural Gas Fired Combined Cycle,1,1,0,0,0,1,8296.9,0.0,518.56,0,0.0,0,10916,0,0,3.77,96,2.0,7.42,pacific_naturalgas,0.444,0.64,0.64,6,6,1.0,1.0
8,8,CA_N_natural_gas_fired_combustion_turbine_1,Natural Gas Fired Combustion Turbine,1,1,0,0,0,1,2248.7,0.0,86.49,0,0.0,0,10999,0,0,5.53,125,3.5,10.7,pacific_naturalgas,0.508,3.78,3.78,1,1,1.0,1.0
9,9,CA_N_natural_gas_fired_combustion_turbine_2,Natural Gas Fired Combustion Turbine,1,1,0,0,0,1,2483.5,0.0,24.84,0,0.0,0,14600,0,0,5.53,125,3.5,10.95,pacific_naturalgas,0.565,3.78,3.78,1,1,1.0,1.0


In [7]:
# # Read demand input data and record parameters
# demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Load_data.csv")))
# # Value of lost load (cost of involuntary non-served energy)
demand_inputs = CSV.read("./Western_interconnect_full_run_10d/Load_data.csv", DataFrame)
VOLL = demand_inputs.Voll[1]
  # Set of price responsive demand (non-served energy) segments
S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_Segment))) 
#NOTE:  collect(skipmising(input)) is needed here in several spots because the demand inputs are not 'square' (different column lengths)

  # Data frame for price responsive demand segments (nse)
  # NSE_Cost = opportunity cost per MWh of demand curtailment
  # NSE_Max = maximum % of demand that can be curtailed in each hour
  # Note that nse segment 1 = involuntary non-served energy (load shedding) at $9000/MWh
  # and segment 2 = one segment of voluntary price responsive demand at $600/MWh (up to 7.5% of demand)
nse = DataFrame(Segment=S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs.Cost_of_Demand_Curtailment_per_MW)),
                NSE_Max = collect(skipmissing(demand_inputs.Max_Demand_Curtailment)))

  # Set of sequential hours per sub-period
hours_per_period = convert(Int64, demand_inputs.Timesteps_per_Rep_Period[1])
  # Set of time sample sub-periods (e.g. sample days or weeks)
P = convert(Array{Int64}, 1:demand_inputs.Rep_Periods[1])
  # Sub period cluster weights = number of hours represented by each sample period
W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
  # Set of all time steps
T = convert(Array{Int64}, demand_inputs.Time_Index)
  # Create vector of sample weights, representing how many hours in the year
  # each hour in each sample period represents
sample_weight = zeros(Float64, size(T,1))
t=1
for p in P
    for h in 1:hours_per_period
        sample_weight[t] = W[p]/hours_per_period
        t=t+1
    end
end

  # Set of zones 
Z = convert(Array{Int64}, 1:6)

  # Load/demand time series by zone (TxZ array)
demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3, :Load_MW_z4, :Load_MW_z5, :Load_MW_z6 );
# Uncomment this line to explore the data if you wish:
show(demand, allrows=true, allcols=true)

240×6 DataFrame
 Row │ Load_MW_z1  Load_MW_z2  Load_MW_z3  Load_MW_z4  Load_MW_z5  Load_MW_z6 
     │ Int64       Int64       Int64       Int64       Int64       Int64      
─────┼────────────────────────────────────────────────────────────────────────
   1 │       9669       15030        9271       15348       18903        9625
   2 │       9074       14102        8994       14972       18128        9811
   3 │       8781       13645        8902       14873       17787        9708
   4 │       8576       13326        8844       14894       17620        9621
   5 │       8502       13214        8923       15101       17718        9661
   6 │       8641       13431        9193       15578       18287        9835
   7 │       8954       13930        9472       15759       19287        9811
   8 │       8903       13842        9754       15889       19935       10088
   9 │       9269       14412       10041       15967       20793       10382
  10 │       9709       15097       10291    

In [11]:
# Read generator capacity factors by hour (used for variable renewables)
  # There is one column here for each resource (row) in the generators DataFrame
#variability = DataFrame(CSV.File(joinpath(inputs_path, "Generators_variability.csv")))
variability = CSV.read("./Western_interconnect_full_run_10d/Generators_variability.csv", DataFrame)
  # Drop the first column with row indexes, as these are unecessary
variability = variability[:,2:ncol(variability)];
# Uncomment this line to explore the data if you wish:
# show(variability, allrows=true, allcols=true)

In [13]:
variability

Row,CA_N_batteries_1,CA_N_biomass_1,CA_N_conventional_hydroelectric_1,CA_N_geothermal_1,CA_N_hydroelectric_pumped_storage_1,CA_N_natural_gas_fired_combined_cycle_1,CA_N_natural_gas_fired_combined_cycle_2,CA_N_natural_gas_fired_combustion_turbine_1,CA_N_natural_gas_fired_combustion_turbine_2,CA_N_natural_gas_steam_turbine_1,CA_N_onshore_wind_turbine_1,CA_N_other_peaker_1,CA_N_small_hydroelectric_1,CA_N_solar_photovoltaic_1,CA_S_batteries_1,CA_S_biomass_1,CA_S_conventional_hydroelectric_1,CA_S_geothermal_1,CA_S_hydroelectric_pumped_storage_1,CA_S_natural_gas_fired_combined_cycle_1,CA_S_natural_gas_fired_combined_cycle_2,CA_S_natural_gas_fired_combustion_turbine_1,CA_S_natural_gas_fired_combustion_turbine_2,CA_S_natural_gas_steam_turbine_1,CA_S_onshore_wind_turbine_1,CA_S_other_peaker_1,CA_S_small_hydroelectric_1,CA_S_solar_photovoltaic_1,WECC_N_batteries_1,WECC_N_biomass_1,WECC_N_conventional_hydroelectric_1,WECC_N_conventional_steam_coal_1,WECC_N_geothermal_1,WECC_N_natural_gas_fired_combined_cycle_1,WECC_N_natural_gas_fired_combined_cycle_2,WECC_N_natural_gas_fired_combustion_turbine_1,WECC_N_natural_gas_fired_combustion_turbine_2,WECC_N_natural_gas_steam_turbine_1,WECC_N_onshore_wind_turbine_1,WECC_N_other_peaker_1,WECC_N_solar_photovoltaic_1,WECC_NMAZ_batteries_1,WECC_NMAZ_biomass_1,WECC_NMAZ_conventional_hydroelectric_1,WECC_NMAZ_conventional_steam_coal_1,WECC_NMAZ_geothermal_1,WECC_NMAZ_hydroelectric_pumped_storage_1,WECC_NMAZ_natural_gas_fired_combined_cycle_1,WECC_NMAZ_natural_gas_fired_combined_cycle_2,WECC_NMAZ_natural_gas_fired_combustion_turbine_1,WECC_NMAZ_natural_gas_fired_combustion_turbine_2,WECC_NMAZ_natural_gas_steam_turbine_1,WECC_NMAZ_nuclear_1,WECC_NMAZ_onshore_wind_turbine_1,WECC_NMAZ_other_peaker_1,WECC_NMAZ_solar_photovoltaic_1,WECC_PNW_batteries_1,WECC_PNW_biomass_1,WECC_PNW_conventional_hydroelectric_1,WECC_PNW_geothermal_1,WECC_PNW_hydroelectric_pumped_storage_1,WECC_PNW_natural_gas_fired_combined_cycle_1,WECC_PNW_natural_gas_fired_combined_cycle_2,WECC_PNW_natural_gas_fired_combustion_turbine_1,WECC_PNW_natural_gas_fired_combustion_turbine_2,WECC_PNW_natural_gas_steam_turbine_1,WECC_PNW_nuclear_1,WECC_PNW_onshore_wind_turbine_1,WECC_PNW_other_peaker_1,WECC_PNW_solar_photovoltaic_1,WECC_WYCO_batteries_1,WECC_WYCO_biomass_1,WECC_WYCO_conventional_hydroelectric_1,WECC_WYCO_conventional_steam_coal_1,WECC_WYCO_hydroelectric_pumped_storage_1,WECC_WYCO_natural_gas_fired_combined_cycle_1,WECC_WYCO_natural_gas_fired_combined_cycle_2,WECC_WYCO_natural_gas_fired_combustion_turbine_1,WECC_WYCO_natural_gas_fired_combustion_turbine_2,WECC_WYCO_natural_gas_steam_turbine_1,WECC_WYCO_onshore_wind_turbine_1,WECC_WYCO_other_peaker_1,WECC_WYCO_solar_photovoltaic_1,CA_N_naturalgas_ccavgcf_moderate_0,CA_N_naturalgas_ctavgcf_moderate_0,CA_N_battery_moderate_0,CA_N_nuclear_nuclear_moderate_0,CA_N_naturalgas_ccs100_moderate_0,CA_N_landbasedwind_class6_moderate_1,CA_N_landbasedwind_class6_moderate_2,CA_N_landbasedwind_class6_moderate_3,CA_N_landbasedwind_class6_moderate_4,CA_N_landbasedwind_class6_moderate_5,CA_N_landbasedwind_class6_moderate_6,CA_N_landbasedwind_class6_moderate_7,CA_N_landbasedwind_class6_moderate_8,CA_N_landbasedwind_class6_moderate_9,CA_N_landbasedwind_class6_moderate_10,CA_N_landbasedwind_class6_moderate_11,CA_N_landbasedwind_class6_moderate_12,⋯
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1,1,0.2

In [15]:
using CSV
using DataFrames

# Read fuels data
fuels_data = CSV.read("./Western_interconnect_full_run_10d/Fuels_data.csv", DataFrame)

Row,Fuel,Cost_per_MMBtu,CO2_content_tons_per_MMBtu,fuel_indices
,String31,Float64,Float64,Int64
1,None,0.0,0.0,1
2,pacific_naturalgas,4.35,0.05306,2
3,rockies_coal,1.67,0.09552,3
4,rockies_naturalgas,4.2,0.05306,4
5,rockies_uranium,0.73,0.0,5
6,pacific_uranium,0.73,0.0,6
7,pacific_naturalgas_ccs100,4.35,0.0,7
8,rockies_naturalgas_ccs100,4.2,0.0,8


In [16]:
fuels_data

Row,Fuel,Cost_per_MMBtu,CO2_content_tons_per_MMBtu,fuel_indices
,String31,Float64,Float64,Int64
1,None,0.0,0.0,1
2,pacific_naturalgas,4.35,0.05306,2
3,rockies_coal,1.67,0.09552,3
4,rockies_naturalgas,4.2,0.05306,4
5,rockies_uranium,0.73,0.0,5
6,pacific_uranium,0.73,0.0,6
7,pacific_naturalgas_ccs100,4.35,0.0,7
8,rockies_naturalgas_ccs100,4.2,0.0,8


In [19]:
fuel_types = fuels_data[1:end, 1]

8-element Vector{String31}:
 "None"
 "pacific_naturalgas"
 "rockies_coal"
 "rockies_naturalgas"
 "rockies_uranium"
 "pacific_uranium"
 "pacific_naturalgas_ccs100"
 "rockies_naturalgas_ccs100"

In [20]:
cost = collect(fuels_data[1:end, 2])

8-element Vector{Float64}:
 0.0
 4.35
 1.67
 4.2
 0.73
 0.73
 4.35
 4.2

In [21]:
co2_emission = collect(fuels_data[1:end, 3])

8-element Vector{Float64}:
 0.0
 0.05306
 0.09552
 0.05306
 0.0
 0.0
 0.0
 0.0

In [22]:
fuels = DataFrame(Fuel = fuel_types, Cost_per_MMBtu = cost, CO2_content_tons_per_MMBtu = co2_emission)

Row,Fuel,Cost_per_MMBtu,CO2_content_tons_per_MMBtu
,String31,Float64,Float64
1,None,0.0,0.0
2,pacific_naturalgas,4.35,0.05306
3,rockies_coal,1.67,0.09552
4,rockies_naturalgas,4.2,0.05306
5,rockies_uranium,0.73,0.0
6,pacific_uranium,0.73,0.0
7,pacific_naturalgas_ccs100,4.35,0.0
8,rockies_naturalgas_ccs100,4.2,0.0


In [29]:
 # Read network data
network = CSV.read("./Western_interconnect_full_run_10d/Network.csv", DataFrame)

Row,Network_zones,Network_Lines,z1,z2,z3,z4,z5,z6,Line_Max_Flow_MW,Line_Min_Flow_MW,transmission_path_name,Line_Reinforcement_Cost_per_MWyr,Line_Reinforcement_Cost_per_MW,Line_Loss_Percentage,Line_Max_Reinforcement_MW
,String3?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String31,Float64,Float64,Float64,Float64
1,z1,1,1,-1,0,0,0,0,3675.0,3675.0,CA_N_to_CA_S,59417.0,1.22664e6,0.0649,2572.0
2,z2,2,1,0,-1,0,0,0,100.0,100.0,CA_N_to_WECC_N,62090.0,1.28182e6,0.0649,70.0
3,z3,3,1,0,0,0,-1,0,3675.0,3675.0,CA_N_to_WECC_PNW,81821.0,1.68915e6,0.0649,2572.0
4,z4,4,0,1,-1,0,0,0,1400.0,1400.0,CA_S_to_WECC_N,52867.0,1.09141e6,0.0649,980.0
5,z5,5,0,1,0,-1,0,0,10464.0,10464.0,CA_S_to_WECC_NMAZ,57757.0,1.19237e6,0.0649,7325.0
6,z6,6,0,1,0,0,-1,0,2858.0,2858.0,CA_S_to_WECC_PNW,93397.0,1.92814e6,0.0649,2001.0
7,missing,7,0,0,1,-1,0,0,740.0,740.0,WECC_N_to_WECC_NMAZ,54734.0,1.12997e6,0.0649,518.0
8,missing,8,0,0,1,0,-1,0,5150.0,5150.0,WECC_N_to_WECC_PNW,76658.0,1.58256e6,0.0649,3605.0
9,missing,9,0,0,1,0,0,-1,4150.0,4150.0,WECC_N_to_WECC_WYCO,39873.0,823153.0,0.0649,2905.0


In [30]:
#Again, there is a lot of entries in here we will not use (formatted for GenX inputs), so let's select what we want
  # Array of network zones (z1, z2, z3)
zones = collect(skipmissing(network.Network_zones))

6-element Vector{String3}:
 "z1"
 "z2"
 "z3"
 "z4"
 "z5"
 "z6"

In [32]:
# Network map showing lines connecting zones
lines = select(network[1:6,:], 
    :Network_Lines, :z1, :z2, :z3, :z4, :z5, :z6 ,
    :Line_Max_Flow_MW, :Line_Min_Flow_MW, :Line_Loss_Percentage, 
    :Line_Max_Reinforcement_MW, :Line_Reinforcement_Cost_per_MWyr)

Row,Network_Lines,z1,z2,z3,z4,z5,z6,Line_Max_Flow_MW,Line_Min_Flow_MW,Line_Loss_Percentage,Line_Max_Reinforcement_MW,Line_Reinforcement_Cost_per_MWyr
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,1,1,-1,0,0,0,0,3675.0,3675.0,0.0649,2572.0,59417.0
2,2,1,0,-1,0,0,0,100.0,100.0,0.0649,70.0,62090.0
3,3,1,0,0,0,-1,0,3675.0,3675.0,0.0649,2572.0,81821.0
4,4,0,1,-1,0,0,0,1400.0,1400.0,0.0649,980.0,52867.0
5,5,0,1,0,-1,0,0,10464.0,10464.0,0.0649,7325.0,57757.0
6,6,0,1,0,0,-1,0,2858.0,2858.0,0.0649,2001.0,93397.0


In [35]:
 # Add fixed O&M costs for lines = 1/20 of reinforcement cost
lines.Line_Fixed_Cost_per_MWyr = lines.Line_Reinforcement_Cost_per_MWyr./20
  # Set of all lines
L = convert(Array{Int64}, lines.Network_Lines);
# Uncomment this line to explore the data if you wish:
show(lines, allrows=true, allcols=true)

6×13 DataFrame
 Row │ Network_Lines  z1     z2     z3     z4     z5     z6     Line_Max_Flow_MW  Line_Min_Flow_MW  Line_Loss_Percentage  Line_Max_Reinforcement_MW  Line_Reinforcement_Cost_per_MWyr  Line_Fixed_Cost_per_MWyr 
     │ Int64          Int64  Int64  Int64  Int64  Int64  Int64  Float64           Float64           Float64               Float64                    Float64                           Float64                  
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │             1      1     -1      0      0      0      0            3675.0            3675.0                0.0649                     2572.0                           59417.0                   2970.85
   2 │             2      1      0     -1      0      0      0             100.0             100.0                0.0649                       70.0   

In [37]:

# Calculate generator (and storage) total variable costs, start-up costs, 
# and associated CO2 per MWh and per start
generators.Var_Cost = zeros(Float64, size(G,1))
generators.CO2_Rate = zeros(Float64, size(G,1))
generators.Start_Cost = zeros(Float64, size(G,1))
generators.CO2_Per_Start = zeros(Float64, size(G,1))
println(size(G,1))

297


In [39]:
for g in G
    # Variable cost (/MWh) = variable O&M (/MWh) + fuel cost ($/MMBtu) * heat rate (MMBtu/MWh)
    generators.Var_Cost[g] = generators.Var_OM_Cost_per_MWh[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Heat_Rate_MMBTU_per_MWh[g]
    
    # CO2 emissions rate (tCO2/MWh) = fuel CO2 content (tCO2/MMBtu) * heat rate (MMBtu/MWh)
    generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Heat_Rate_MMBTU_per_MWh[g]
    
    # Start-up cost (/start/MW) = start up O&M cost (/start/MW) + fuel cost ($/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.Start_Cost[g] = generators.Start_Cost_per_MW[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Start_Fuel_MMBTU_per_MW[g]
    
    # Start-up CO2 emissions (tCO2/start/MW) = fuel CO2 content (tCO2/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Start_Fuel_MMBTU_per_MW[g]
end
# Note: after this, we don't need the fuels Data Frame again...

In [41]:
# Drop hydropower and biomass plants from generators set for simplicity 
# (these are a small share of total capacity, ~500 MW
# G = intersect(generators.R_ID[.!(generators.HYDRO.==1)],G)
# G = intersect(generators.R_ID[.!(generators.technology.=="Biomass")],G);

In [43]:

# Organize all of our data in one place...
# This code block is unecessary, but after all of the input steps above
# writing it all here helps us see all of the sets and parameter DataFrames.

# SUBSETS
  # By naming convention, all sets are single capital letters
G # Set of all generators
S # Set of all non-served energy (price responsive demand) segments
P  # Set of time sample sub-periods (e.g. sample days or weeks)
W  # Sub period cluster weights = number of periods (days/weeks) represented by each sample period
T  # Set of all time steps 
Z  # Set of zones (by number)
L;  # Set of all lines

In [45]:

# PARAMETER DataFrames
  # By naming convention, all parameter data frames are lowercase
nse        # non-served energy parameters (by s in S)
generators # generation (and storage) parameters (by g in G)
demand     # demand parameters (by t in T)
zones      # network zones (by z in Z)
lines;      # transmission lines (by l in L)
     

In [47]:

#SUBSETS
  # By naming convention, all subsets are UPPERCASE

  # Subset of G of all thermal resources subject to unit commitment constraints
UC = intersect(generators.R_ID[generators.Commit.==1], G)
  # Subset of G NOT subject to unit commitment constraints
ED = intersect(generators.R_ID[.!(generators.Commit.==1)], G)
  # Subset of G of all storage resources
STOR = intersect(generators.R_ID[generators.STOR.>=1], G)
  # Subset of G of all variable renewable resources
VRE = intersect(generators.R_ID[generators.VRE.==1], G)
  # Subset of all new build resources
NEW = intersect(generators.R_ID[generators.New_Build.==1], G)
  # Subset of all existing resources
OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)
#   # Subset of all RPS qualifying resources
# RPS = intersect(generators.R_ID[generators.RPS.==1], G);

# Notes: findall(x->x in A, B) returns the intersection of two vectors A and B

101-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 286
 287
 288
 289
 290
 291
 292
 293
 294
 295
 296
 297

In [49]:

# LP model using HiGHS solver
Expansion_Model =  Model(HiGHS.Optimizer);

In [50]:
# DECISION VARIABLES
  # By naming convention, all decision variables start with v and then are in UPPER_SNAKE_CASE

# Capacity decision variables
@variables(Expansion_Model, begin
        vCAP[g in G]            >= 0     # power capacity (MW)
        vRET_CAP[g in OLD]      >= 0     # retirement of power capacity (MW)
        vNEW_CAP[g in NEW]      >= 0     # new build power capacity (MW)
        
        vE_CAP[g in STOR]       >= 0     # storage energy capacity (MWh)
        vRET_E_CAP[g in intersect(STOR, OLD)]   >= 0     # retirement of storage energy capacity (MWh)
        vNEW_E_CAP[g in intersect(STOR, NEW)]   >= 0     # new build storage energy capacity (MWh)
        
        vT_CAP[l in L]          >= 0     # transmission capacity (MW)
        vRET_T_CAP[l in L]      >= 0     # retirement of transmission capacity (MW)
        vNEW_T_CAP[l in L]      >= 0     # new build transmission capacity (MW)
end)

# Set upper bounds on capacity for renewable resources 
# (which are limited in each resource 'cluster')
for g in NEW[generators[NEW,:Max_Cap_MW].>0]
    set_upper_bound(vNEW_CAP[g], generators.Max_Cap_MW[g])
end

# Set upper bounds on transmission capacity expansion
for l in L
    set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
end

# Operational decision variables
@variables(Expansion_Model, begin
        vGEN[T,G]       >= 0  # Power generation (MW)
        vCHARGE[T,STOR] >= 0  # Power charging (MW)
        vSOC[T,STOR]    >= 0  # Energy storage state of charge (MWh)
        vNSE[T,S,Z]     >= 0  # Non-served energy/demand curtailment (MW)
        vFLOW[T,L]      # Transmission line flow (MW); 
          # note line flow is positive if flowing
          # from source node (indicated by 1 in zone column for that line) 
          # to sink node (indicated by -1 in zone column for that line); 
          # flow is negative if flowing from sink to source.
end)

(2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
And data, a 240×297 Matrix{VariableRef}:
 vGEN[1,1]    vGEN[1,2]    vGEN[1,3]    …  vGEN[1,296]    vGEN[1,297]
 vGEN[2,1]    vGEN[2,2]    vGEN[2,3]       vGEN[2,296]    vGEN[2,297]
 vGEN[3,1]    vGEN[3,2]    vGEN[3,3]       vGEN[3,296]    vGEN[3,297]
 vGEN[4,1]    vGEN[4,2]    vGEN[4,3]       vGEN[4,296]    vGEN[4,297]
 vGEN[5,1]    vGEN[5,2]    vGEN[5,3]       vGEN[5,296]    vGEN[5,297]
 vGEN[6,1]    vGEN[6,2]    vGEN[6,3]    …  vGEN[6,296]    vGEN[6,297]
 vGEN[7,1]    vGEN[7,2]    vGEN[7,3]       vGEN[7,296]    vGEN[7,297]
 vGEN[8,1]    vGEN[8,2]    vGEN[8,3]       vGEN[8,296]    vGEN[8,297]
 vGEN[9,1]    vGEN[9,2]    vGEN[9,3]       vGEN[9,296]    vGEN[9,297]
 vGEN[10,1]   vGEN[10,2]   vGEN[10,3]      vGEN[10,296]   v

In [52]:

# CONSTRAINTS
  # By naming convention, all constraints start with c and then are TitleCase

# (1) Supply-demand balance constraint for all time steps and zones
@constraint(Expansion_Model, cDemandBalance[t in T, z in Z], 
        sum(vGEN[t,g] for g in intersect(generators[generators.Zone.==z,:R_ID],G)) +
        sum(vNSE[t,s,z] for s in S) - 
        sum(vCHARGE[t,g] for g in intersect(generators[generators.Zone.==z,:R_ID],STOR)) -
        demand[t,z] - 
        sum(lines[l,Symbol(string("z",z))] * vFLOW[t,l] for l in L) == 0
);

In [53]:

# (2-6) Capacitated constraints:
@constraints(Expansion_Model, begin
# (2) Max power constraints for all time steps and all generators/storage
    cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t,g]*vCAP[g]
# (3) Max charge constraints for all time steps and all storage resources
    cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]
# (4) Max state of charge constraints for all time steps and all storage resources
    cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]
# (5) Max non-served energy constraints for all time steps and all segments and all zones
    cMaxNSE[t in T, s in S, z in Z], vNSE[t,s,z] <= nse.NSE_Max[s]*demand[t,z]
# (6a) Max flow constraints for all time steps and all lines
    cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]
# (6b) Min flow constraints for all time steps and all lines
    cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]
end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
And data, a 240×297 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cMaxPower[1,1] : -vCAP[1] + vGEN[1,1] <= 0      …  cMaxPower[1,297] : -0.265 vCAP[297] + vGEN[1,297] <= 0
 cMaxPower[2,1] : -vCAP[1] + vGEN[2,1] <= 0         cMaxPower[2,297] : -0.154 vCAP[297] + vGEN[2,297] <= 0
 cMaxPower[3,1] : -vCAP[1] + vGEN[3,1] <= 0         cMaxPower[3,297] : -0.088 vCAP[297] + vGEN[3,297] <= 0
 cMaxPower[4,1] : -vCAP[1] + vGEN[4,1] <= 0         cMaxPower[4,297] : -0.087 vCAP[297] 

In [54]:

# (7-9) Total capacity constraints:
@constraints(Expansion_Model, begin
# (7a) Total capacity for existing units
    cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]
# (7b) Total capacity for new units
    cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]
        
# (8a) Total energy storage capacity for existing units
    cCapEnergyOld[g in intersect(STOR, OLD)], 
        vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]
# (8b) Total energy storage capacity for existing units
    cCapEnergyNew[g in intersect(STOR, NEW)], 
        vE_CAP[g] == vNEW_E_CAP[g]
        
# (9) Total transmission capacity
    cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]
end)
     

(1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
And data, a 101-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 cCapOld[1] : vCAP[1] + vRET_CAP[1] == 2613.1
 cCapOld[2] : vCAP[2] + vRET_CAP[2] == 250.4
 cCapOld[3] : vCAP[3] + vRET_CAP[3] == 7519.1
 cCapOld[4] : vCAP[4] + vRET_CAP[4] == 757.8
 cCapOld[5] : vCAP[5] + vRET_CAP[5] == 2276.9
 cCapOld[6] : vCAP[6] + vRET_CAP[6] == 3112.8
 cCapOld[7] : vCAP[7] + vRET_CAP[7] == 8296.9
 cCapOld[8] : vCAP[8] + vRET_CAP[8] == 2248.7
 cCapOld[9] : vCAP[9] + vRET_CAP[9] == 2483.5
 cCapOld[10] : vCAP[10] + vRET_CAP[10] == 2.9
 cCapOld[11] : vCAP[11] + vRET_CAP[11] == 1486.7
 c

In [55]:

# Because we are using time domain reduction via sample periods (days or weeks),
# we must be careful with time coupling constraints at the start and end of each
# sample period. 

 # First we record a subset of time steps that begin a sub period 
 # (these will be subject to 'wrapping' constraints that link the start/end of each period)
STARTS = 1:hours_per_period:maximum(T)        
 # Then we record all time periods that do not begin a sub period 
# (these will be subject to normal time couping constraints, looking back one period)
INTERIORS = setdiff(T,STARTS)

# (10-12) Time coupling constraints
@constraints(Expansion_Model, begin
    # (10a) Ramp up constraints, normal
    cRampUp[t in INTERIORS, g in G], 
        vGEN[t,g] - vGEN[t-1,g] <= generators.Ramp_Up_Percentage[g]*vCAP[g]
    # (10b) Ramp up constraints, sub-period wrapping
    cRampUpWrap[t in STARTS, g in G], 
        vGEN[t,g] - vGEN[t+hours_per_period-1,g] <= generators.Ramp_Up_Percentage[g]*vCAP[g]    
    
    # (11a) Ramp down, normal
    cRampDown[t in INTERIORS, g in G], 
        vGEN[t-1,g] - vGEN[t,g] <= generators.Ramp_Dn_Percentage[g]*vCAP[g] 
    # (11b) Ramp down, sub-period wrapping
    cRampDownWrap[t in STARTS, g in G], 
        vGEN[t+hours_per_period-1,g] - vGEN[t,g] <= generators.Ramp_Dn_Percentage[g]*vCAP[g]     
   
    # (12a) Storage state of charge, normal
    cSOC[t in INTERIORS, g in STOR], 
        vSOC[t,g] == vSOC[t-1,g] + generators.Eff_Up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_Down[g]
    # (12a) Storage state of charge, wrapping
    cSOCWrap[t in STARTS, g in STOR], 
        vSOC[t,g] == vSOC[t+hours_per_period-1,g] + generators.Eff_Up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_Down[g]
end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  288, 289, 290, 291, 292, 293, 294, 295, 296, 297]
And data, a 230×297 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cRampUp[2,1] : -vCAP[1] - vGEN[1,1] + vGEN[2,1] <= 0        …  cRampUp[2,297] : -vGEN[1,297] + vGEN[2,297] <= 0
 cRampUp[3,1] : -vCAP[1] - vGEN[2,1] + vGEN[3,1] <= 0           cRampUp[3,297] : -vGEN[2,297] + vGEN[3,297] <= 0
 cRampUp[4,1] : -vCAP[1] - vGEN[3,1] + vGEN[4,1] <= 0           cRampUp[4,297] : -vGEN[3,297] + vGEN[4,297] <= 0
 cRampUp[5,1] : -vCAP[1] - vGEN[4,1] + vGEN[5,1] <= 0           cRamp

In [56]:

# The objective function is to minimize the sum of fixed costs associated with
# capacity decisions and variable costs associated with operational decisions
unregister(Expansion_Model, :eFixedCostsGeneration)
unregister(Expansion_Model, :eFixedCostsStorage)
unregister(Expansion_Model, :eFixedCostsTransmission)
unregister(Expansion_Model, :eVariableCosts)
unregister(Expansion_Model, :eNSECosts)

# Create expressions for each sub-component of the total cost (for later retrieval)
@expression(Expansion_Model, eFixedCostsGeneration,
     # Fixed costs for total capacity 
    sum(generators.Fixed_OM_Cost_per_MWyr[g]*vCAP[g] for g in G) +
     # Investment cost for new capacity
    sum(generators.Inv_Cost_per_MWyr[g]*vNEW_CAP[g] for g in NEW)
)
@expression(Expansion_Model, eFixedCostsStorage,
     # Fixed costs for total storage energy capacity 
    sum(generators.Fixed_OM_Cost_per_MWhyr[g]*vE_CAP[g] for g in STOR) + 
     # Investment costs for new storage energy capacity
    sum(generators.Inv_Cost_per_MWhyr[g]*vNEW_E_CAP[g] for g in intersect(STOR, NEW))
)
@expression(Expansion_Model, eFixedCostsTransmission,
     # Investment and fixed O&M costs for transmission lines
    sum(lines.Line_Fixed_Cost_per_MWyr[l]*vT_CAP[l] +
        lines.Line_Reinforcement_Cost_per_MWyr[l]*vNEW_T_CAP[l] for l in L)
)
@expression(Expansion_Model, eVariableCosts,
     # Variable costs for generation, weighted by hourly sample weight
    sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G)
)
@expression(Expansion_Model, eNSECosts,
     # Non-served energy costs
    sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)
)

@objective(Expansion_Model, Min,
    eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
    eVariableCosts + eNSECosts
);

In [57]:
@time optimize!(Expansion_Model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-03, 4e+00]
  Cost   [4e+00, 6e+05]
  Bound  [2e+00, 5e+05]
  RHS    [3e+00, 6e+04]
Presolving model
187086 rows, 71461 cols, 554600 nonzeros  0s
177009 rows, 67261 cols, 533006 nonzeros  1s
Presolve : Reductions: rows 177009(-56591); columns 67261(-17145); elements 533006(-140960)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1446(7.89219e+06); Du: 0(6.12098e-10) 2s
       8955     1.1588256095e+09 Pr: 4084(1.31266e+07); Du: 0(2.21022e-06) 7s
      13525     2.0159270082e+09 Pr: 4360(2.31307e+07); Du: 0(2.09894e-06) 13s
      15549     2.0669132203e+09 Pr: 5195(2.63294e+08); Du: 0(1.97791e-06) 18s
      16391     2.6695379106e+09 Pr: 5428(5.34433e+07); Du: 0(1.78963e-06) 23s
      18143     3.1711102762e+09 Pr: 4467(4.95376e+07); Du: 0(1.66435e-06) 29s

In [ ]:
# Record generation capacity and energy results
generation = zeros(size(G,1))
for i in 1:size(G,1)
    # Note that total annual generation is sumproduct of sample period weights and hourly sample period generation 
    generation[i] = sum(sample_weight.*value.(vGEN)[:,G[i]].data) 
end

# Total annual demand is sumproduct of sample period weights and hourly sample period demands
total_demand = sum(sum.(eachcol(sample_weight.*demand)))
# Maximum aggregate demand is the maximum of the sum of total concurrent demand in each hour
peak_demand = maximum(sum(eachcol(demand)))
MWh_share = generation./total_demand.*100
cap_share = value.(vCAP).data./peak_demand.*100
generator_results = DataFrame(
    ID = G, 
    Resource = generators.Resource[G],
    Zone = generators.zone[G],
    Total_MW = value.(vCAP).data,
    Start_MW = generators.Existing_Cap_MW[G],
    Change_in_MW = value.(vCAP).data.-generators.Existing_Cap_MW[G],
    Percent_MW = cap_share,
    GWh = generation/1000,
    Percent_GWh = MWh_share
)

# Record energy storage energy capacity results (MWh)
storage_results = DataFrame(
    ID = STOR, 
    Zone = generators.zone[STOR],
    Resource = generators.Resource[STOR],
    Total_Storage_MWh = value.(vE_CAP).data,
    Start_Storage_MWh = generators.Existing_Cap_MWh[STOR],
    Change_in_Storage_MWh = value.(vE_CAP).data.-generators.Existing_Cap_MWh[STOR],
)


# Record transmission capacity results
transmission_results = DataFrame(
    Line = L, 
    Total_Transfer_Capacity = value.(vT_CAP).data,
    Start_Transfer_Capacity = lines.Line_Max_Flow_MW,
    Change_in_Transfer_Capacity = value.(vT_CAP).data.-lines.Line_Max_Flow_MW,
)


## Record non-served energy results by segment and zone
num_segments = maximum(S)
num_zones = maximum(Z)
nse_results = DataFrame(
    Segment = zeros(num_segments*num_zones),
    Zone = zeros(num_segments*num_zones),
    NSE_Price = zeros(num_segments*num_zones),
    Max_NSE_MW = zeros(num_segments*num_zones),
    Total_NSE_MWh = zeros(num_segments*num_zones),
    NSE_Percent_of_Demand = zeros(num_segments*num_zones)
)
i=1
for s in S
    for z in Z
        nse_results.Segment[i]=s
        nse_results.Zone[i]=z
        nse_results.NSE_Price[i]=nse.NSE_Cost[s]
        nse_results.Max_NSE_MW[i]=maximum(value.(vNSE)[:,s,z].data)
        nse_results.Total_NSE_MWh[i]=sum(sample_weight.*value.(vNSE)[:,s,z].data)
        nse_results.NSE_Percent_of_Demand[i]=sum(sample_weight.*value.(vNSE)[:,s,z].data)/total_demand*100
        i=i+1
    end
end

# Record costs by component (in million dollars)
 # Note: because each expression evaluates to a single value, 
 # value.(JuMPObject) returns a numerical value, not a DenseAxisArray;
 # We thus do not need to use the .data extension here to extract numeric values
cost_results = DataFrame(
    Total_Costs = objective_value(Expansion_Model)/10^6,
    Fixed_Costs_Generation = value.(eFixedCostsGeneration)/10^6,
    Fixed_Costs_Storage = value.(eFixedCostsStorage)/10^6,
    Fixed_Costs_Transmission = value.(eFixedCostsTransmission)/10^6,
    Variable_Costs = value.(eVariableCosts)/10^6,
    NSE_Costs = value.(eNSECosts)/10^6
);

In [ ]:
# Output path [set path to desired output directory here]
outpath = "results_western_4weeks" #/YOUR/PATH/HERE

# If output directory does not exist, create it
if !(isdir(outpath))
    mkdir(outpath)
end

CSV.write(joinpath(outpath, "generator_results.csv"), generator_results)
CSV.write(joinpath(outpath, "storage_results.csv"), storage_results)
CSV.write(joinpath(outpath, "transmission_results.csv"), transmission_results)
CSV.write(joinpath(outpath, "nse_results.csv"), nse_results)
CSV.write(joinpath(outpath, "cost_results.csv"), cost_results);